In [33]:
import os
import pickle
import click
import mlflow
from mlflow.tracking import MlflowClient

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

"""
For Q3 (train model with autolog), this notebook is for experimenting
updates to the train.py script.
"""

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


# @click.command()
# @click.option(
#     "--data_path",
#     default="./output",
#     help="Location where the processed NYC taxi trip data was saved"
# )

In [34]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

# @click.command()
# @click.option(
#     "--data_path",
#     default="./output",
#     help="Location where the processed NYC taxi trip data was saved"
# )

In [35]:
data_path = "./output"
def run_train(data_path: str):
    
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    mlflow.sklearn.autolog()

    with mlflow.start_run():
    
        rf = RandomForestRegressor(max_depth=10, random_state=0)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
    
        rmse = mean_squared_error(y_val, y_pred, squared=False)

In [36]:
run_train(data_path)

2024/05/26 16:58:13 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/opt/miniconda3/envs/mlops-dtc/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
for x in mlflow.search_experiments():
    print(x)

<Experiment: artifact_location='/Users/cj/Documents/Projects/zoomcamp-homework/mlops/mlflow_hw_2/homework/mlruns/1', creation_time=1716701523249, experiment_id='1', last_update_time=1716701523249, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>
<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716701008608, experiment_id='0', last_update_time=1716701008608, lifecycle_stage='active', name='Default', tags={}>


In [11]:
from mlflow.tracking import MlflowClient
client = MlflowClient("http://127.0.0.1:5000")

In [15]:
client.search_registered_models()

[]

In [31]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id

run_id

'a7f86290778645629acf4b816f4b5892'

In [32]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='taxi-model'
)

Successfully registered model 'taxi-model'.
Created version '1' of model 'taxi-model'.


<ModelVersion: aliases=[], creation_timestamp=1716703904310, current_stage='None', description=None, last_updated_timestamp=1716703904310, name='taxi-model', run_id='a7f86290778645629acf4b816f4b5892', run_link=None, source='/Users/cj/Documents/Projects/zoomcamp-homework/mlops/mlflow_hw_2/homework/mlruns/1/a7f86290778645629acf4b816f4b5892/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=1>